In [2]:
from numpy import mean
from numpy import std
from sklearn.svm import SVC
from sklearn.ensemble import StackingClassifier
from matplotlib import pyplot

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [5]:
Abhi = pd.read_csv("/content/gdrive/MyDrive/Datasets/Edited-mpxj 8.0 - mpxj 8.0.csv")

In [6]:
Abhi['change'] = Abhi['change'].map({'YES': 1, 'NO': 0})

In [7]:
Abhi.shape

(821, 8)

In [8]:
Abhi.head()

,CBO,NOC,RFC,LOC,DIT,LCOM,WMC,change
0,1,0,5,29,1,60,5,0
1,0,0,10,3,3,0,0,0
2,1,0,8,43,1,70,8,0
3,2,8,21,792,1,0,5,0
4,13,0,23,292,1,83,44,1


In [9]:
X = Abhi.drop('change', axis=1)
y = Abhi['change']

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20)

In [11]:
def get_stacking():
	# define the base models
	level0 = list()
	level0.append(('sig', SVC(probability=True,kernel='sigmoid')))
	level0.append(('lin', SVC(probability=True,kernel='linear')))
	level0.append(('poly', SVC(probability=True,kernel='poly')))
	level0.append(('rbf', SVC(probability=True,kernel='rbf')))
	# define meta learner model
	level1 = SVC(probability=True,kernel='rbf')
	# define the stacking ensemble
	model = StackingClassifier(estimators=level0, final_estimator=level1)
	return model

In [12]:
def get_models():
	models = dict()
	#models['lin'] = SVC(kernel='linear')
	#models['sig'] = SVC(kernel='sigmoid')
	#models['poly'] = SVC(kernel='poly')
	#models['rbf'] = SVC(kernel='rbf')

	models['stacking'] = get_stacking()
	return models

In [13]:
# evaluate a give model
def evaluate_model(model, X, y):
  model=model.fit(X_train,y_train)
  y_pred = model.predict(X_test)
  return y_pred

In [14]:
models = get_models()

In [15]:
results, names = list(), list()
for name, model in models.items():
    y_pred = evaluate_model(model, X, y)
    results.append(y_pred)
    names.append(name)
    print('>%s %.3f (%.3f)' % (name, mean(y_pred), std(y_pred)))


>stacking 0.109 (0.312)


In [16]:
print(y_pred)

[0 1 0 0 0 0 0 0 0 0 1 0 0 0 1 0 1 1 0 0 1 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0
 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0
 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0
 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 1 0]


In [17]:
from sklearn.metrics import classification_report, confusion_matrix
cm=confusion_matrix(y_test,y_pred)
print(cm)

[[113   4]
 [ 34  14]]


In [18]:
# predict probabilities
from sklearn.metrics import roc_auc_score
pred_prob = model.predict_proba(X_test)
auc_score = roc_auc_score(y_test, pred_prob[:,1])
print("Area Under Curve=")
print(auc_score)

Area Under Curve=
0.6411146723646725


In [19]:
from sklearn.metrics import matthews_corrcoef
mcc=matthews_corrcoef(y_test,y_pred)
print("Matthews correlation coefficient=")
print(mcc)

Matthews correlation coefficient=
0.375111020383352


In [20]:
from sklearn.metrics import balanced_accuracy_score
bac=balanced_accuracy_score(y_test, y_pred)
print("Balanced Accuracy Score=")
print(bac)

Balanced Accuracy Score=
0.6287393162393162


In [21]:
from imblearn.metrics import geometric_mean_score
gmean=geometric_mean_score(y_test, y_pred)
print("Geometric Mean Score=")
print(gmean)

Geometric Mean Score=
0.5307496177060863


In [22]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.77      0.97      0.86       117
           1       0.78      0.29      0.42        48

    accuracy                           0.77       165
   macro avg       0.77      0.63      0.64       165
weighted avg       0.77      0.77      0.73       165

